# 📊 Análisis Acumulativo de N Preguntas - Google Colab [OPTIMIZADO]

Este notebook implementa el análisis acumulativo de múltiples preguntas comparando la efectividad de diferentes modelos de embedding en la recuperación de documentos usando preguntas vs respuestas.

## ⚡ OPTIMIZACIONES INCLUIDAS:
- **Batch Processing**: Procesa múltiples embeddings a la vez (10-50x más rápido)
- **Model Caching**: Carga modelos una sola vez y los reutiliza
- **Checkpointing**: Guarda progreso cada 10 lotes (recuperación ante fallos)
- **Skip RAG Metrics**: Opción para saltar métricas RAG y ahorrar tiempo
- **Pre-filtering**: Filtra preguntas inválidas antes de procesar
- **Memory Management**: Limpia memoria después de cada modelo

## 🎯 Objetivos
- Evaluar múltiples modelos de embedding simultáneamente
- Comparar recuperación usando preguntas vs respuestas aceptadas
- Calcular métricas: Jaccard, nDCG@10, Precision@5, Score Compuesto
- Generar análisis estadístico completo de los resultados

## 📋 Flujo de Trabajo
1. **Configuración**: Cargar configuración automáticamente desde Google Drive (archivo más reciente)
2. **Datos**: Extraer preguntas desde la configuración y descargar archivos parquet
3. **Evaluación**: Ejecutar análisis con GPU usando embeddings pre-calculados
4. **Resultados**: Guardar métricas consolidadas y detalladas
5. **Visualización**: Ver resultados en Streamlit

## 📋 Instrucciones:

1. **Activar GPU**: Runtime → Change runtime type → GPU → T4
2. **Ejecutar todo**: Runtime → Run all (Ctrl+F9) - ¡La configuración se carga automáticamente!
3. **Autorizar Drive**: Cuando se solicite acceso a Google Drive
4. **Monitorear progreso**: Ver barras de progreso y checkpoints

## ✨ Características:
- 🚀 **Aceleración GPU** para procesamiento rápido de queries
- ⚡ **Batch Processing** para 10-50x mayor velocidad
- 💾 **Checkpointing** para recuperación ante fallos
- 📊 **Embeddings reales**: Usa archivos parquet con embeddings pre-calculados
- 📋 **Preguntas desde configuración**: Las preguntas vienen incluidas en el archivo de configuración
- 🔍 **Comparación pregunta vs respuesta**: Solo usa título + contenido de pregunta para queries
- 📈 **Resultados automáticos** guardados en Google Drive

## 🔧 Opciones de Optimización:

### Para análisis más rápido:
1. **Reducir batch_size** en configuración (default: 50, mínimo: 10)
2. **Desactivar reranking** en configuración 
3. **Desactivar métricas RAG** en configuración
4. **Procesar menos modelos** (descomentar línea en celda 8)
5. **Reducir num_questions** en configuración

### Estimación de tiempo:
- **Sin optimizaciones**: ~1 min por pregunta por modelo
- **Con optimizaciones**: ~1-5 seg por pregunta por modelo
- **600 preguntas, 4 modelos**: 
  - Sin optimizar: ~40 horas
  - Optimizado: ~1-4 horas

## 📤 Resultados:
- Se guardan automáticamente en Google Drive (misma carpeta que configuración)
- Checkpoints cada 10 lotes para recuperación
- Vuelve a Streamlit para ver visualizaciones
- Ve a "Resultados Análisis N Preguntas"

In [ ]:
# 📦 Instalación de dependencias
print("📦 Instalando dependencias...")

# Instalar paquetes necesarios
!pip install -q chromadb sentence-transformers numpy pandas scikit-learn tqdm pyarrow
!pip install -q google-api-python-client google-auth-oauthlib google-auth-httplib2
!pip install -q torch openai transformers accelerate

print("✅ Dependencias instaladas correctamente")

In [ ]:
# 📚 Importaciones y configuración inicial
import json
import time
import random
import numpy as np
import pandas as pd
from datetime import datetime
from typing import Dict, List, Any, Optional
import warnings
warnings.filterwarnings('ignore')

# Google Drive
from google.colab import auth, drive
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload, MediaIoBaseDownload
import io

# ML y NLP
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import ndcg_score

# Progress tracking
from tqdm import tqdm

# Verificar GPU
gpu_available = torch.cuda.is_available()
device = torch.device('cuda' if gpu_available else 'cpu')

print("📚 Importaciones completadas")
print(f"🚀 GPU disponible: {gpu_available}")
if gpu_available:
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 Memoria: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print(f"💻 Dispositivo: {device}")

# Configurar API keys usando Google Colab Secrets
import os

# Cargar API key de OpenAI desde Google Colab Secrets
OPENAI_AVAILABLE = False
try:
    from google.colab import userdata
    openai_key = userdata.get('OPENAI_API_KEY')
    if openai_key:
        os.environ['OPENAI_API_KEY'] = openai_key
        print("✅ OpenAI API key loaded from Colab secrets")
        OPENAI_AVAILABLE = True
    else:
        print("❌ No OpenAI API key found in Colab secrets")
        OPENAI_AVAILABLE = False
except Exception as e:
    print(f"❌ Error loading OpenAI API key: {e}")
    OPENAI_AVAILABLE = False

print(f"🔑 OpenAI API: {'✅ Available' if OPENAI_AVAILABLE else '❌ Not available'}")

In [ ]:
# 🔐 Autenticación con Google Drive
print("🔐 Configurando acceso a Google Drive...")

# Autenticar y montar Google Drive
auth.authenticate_user()
drive.mount('/content/drive')

# Configurar servicio de Google Drive API
service = build('drive', 'v3')

print("✅ Google Drive configurado correctamente")
print("📂 Drive montado en: /content/drive")

In [ ]:
# 📥 Cargar configuración desde Google Drive

def get_drive_file_by_name(filename: str):
    """Buscar archivo por nombre en Google Drive."""
    try:
        results = service.files().list(
            q=f"name='{filename}' and trashed=false",
            fields="files(id, name, modifiedTime)"
        ).execute()
        
        files = results.get('files', [])
        if files:
            return files[0]['id']
        return None
    except Exception as e:
        print(f"❌ Error buscando archivo: {e}")
        return None

def get_latest_config_file():
    """Buscar el archivo de configuración más reciente."""
    try:
        print("🔍 Buscando archivo de configuración más reciente...")
        
        # Buscar archivos que empiecen con 'n_questions_config_' y terminen con '.json'
        results = service.files().list(
            q="name contains 'n_questions_config_' and name contains '.json' and trashed=false",
            fields="files(id, name, modifiedTime)",
            orderBy="modifiedTime desc"
        ).execute()
        
        files = results.get('files', [])
        
        if files:
            latest_file = files[0]
            print(f"📄 Archivo más reciente encontrado: {latest_file['name']}")
            print(f"📅 Modificado: {latest_file['modifiedTime']}")
            return latest_file['name']
        else:
            print("❌ No se encontraron archivos de configuración")
            print("💡 Asegúrate de haber creado una configuración desde Streamlit")
            return None
            
    except Exception as e:
        print(f"❌ Error buscando configuraciones: {e}")
        return None

def load_config_from_drive(config_filename: str):
    """Cargar configuración desde Google Drive."""
    print(f"📥 Cargando configuración: {config_filename}")
    
    file_id = get_drive_file_by_name(config_filename)
    if not file_id:
        print(f"❌ No se encontró el archivo: {config_filename}")
        return None
    
    try:
        request = service.files().get_media(fileId=file_id)
        file_io = io.BytesIO()
        downloader = MediaIoBaseDownload(file_io, request)
        
        done = False
        while done is False:
            status, done = downloader.next_chunk()
        
        file_io.seek(0)
        config = json.loads(file_io.read().decode('utf-8'))
        
        print(f"✅ Configuración cargada exitosamente")
        return config
        
    except Exception as e:
        print(f"❌ Error cargando configuración: {e}")
        return None

# 🚀 Obtener archivo de configuración más reciente automáticamente
CONFIG_FILENAME = get_latest_config_file()

if CONFIG_FILENAME:
    print(f"📥 Usando configuración: {CONFIG_FILENAME}")
    config = load_config_from_drive(CONFIG_FILENAME)
    
    if config:
        print("✅ Configuración cargada exitosamente!")
        print(f"📊 Número de preguntas: {config['data_config']['num_questions']}")
        print(f"🤖 Modelos a evaluar: {list(config['model_config']['embedding_models'].keys())}")
        print(f"📈 Métricas incluidas: {len(config['metrics_config']['metrics_included'])}")
    else:
        print("❌ No se pudo cargar la configuración")
        raise FileNotFoundError("Configuration file could not be loaded")
else:
    print("❌ No se encontró ningún archivo de configuración")
    print("🔍 Verifica que hayas creado una configuración desde Streamlit")
    print("💡 Ve a 'Configuración Análisis N Preguntas' y crea una nueva configuración")
    raise FileNotFoundError("No configuration files found")

In [ ]:
# 📥 Cargar archivo de preguntas desde configuración

def load_questions_from_config(config):
    """Cargar preguntas desde la configuración."""
    print("📥 Extrayendo preguntas desde la configuración...")
    
    # Verificar si la configuración tiene preguntas incluidas
    if 'questions_data' in config:
        questions_data = config['questions_data']
        print(f"✅ Encontradas {len(questions_data)} preguntas en la configuración")
        
        # Procesar preguntas al formato esperado
        questions = []
        for i, qa_item in enumerate(questions_data):
            question = {
                'id': f"config_q_{i}",
                'title': qa_item.get('title', 'Sin título'),
                'content': qa_item.get('question_content', qa_item.get('question', '')),
                'accepted_answer': qa_item.get('accepted_answer', 'Sin respuesta'),
                'ms_links': qa_item.get('ms_links', []),
                'tags': qa_item.get('tags', []),
                'metadata': qa_item
            }
            questions.append(question)
        
        return questions
    
    # Si no hay preguntas en la config, intentar cargar desde Drive
    elif 'questions_file' in config:
        questions_filename = config['questions_file']
        print(f"📥 Intentando cargar preguntas desde: {questions_filename}")
        
        file_id = get_drive_file_by_name(questions_filename)
        if not file_id:
            print(f"❌ No se encontró el archivo: {questions_filename}")
            return []
        
        try:
            request = service.files().get_media(fileId=file_id)
            file_io = io.BytesIO()
            downloader = MediaIoBaseDownload(file_io, request)
            
            done = False
            while done is False:
                status, done = downloader.next_chunk()
            
            file_io.seek(0)
            questions_data = json.loads(file_io.read().decode('utf-8'))
            
            if isinstance(questions_data, list):
                questions = []
                for i, qa_item in enumerate(questions_data):
                    question = {
                        'id': f"file_q_{i}",
                        'title': qa_item.get('title', 'Sin título'),
                        'content': qa_item.get('question_content', qa_item.get('question', '')),
                        'accepted_answer': qa_item.get('accepted_answer', 'Sin respuesta'),
                        'ms_links': qa_item.get('ms_links', []),
                        'tags': qa_item.get('tags', []),
                        'metadata': qa_item
                    }
                    questions.append(question)
                
                print(f"✅ {len(questions)} preguntas cargadas desde archivo")
                return questions
            else:
                print("❌ Formato de archivo de preguntas no válido")
                return []
                
        except Exception as e:
            print(f"❌ Error cargando archivo de preguntas: {e}")
            return []
    
    else:
        print("❌ No se encontraron preguntas en la configuración")
        print("💡 La configuración debe incluir 'questions_data' o 'questions_file'")
        return []

def download_embeddings_from_drive():
    """Descargar archivos de embeddings desde Google Drive."""
    print("📥 Preparando descarga de archivos de embeddings...")
    
    # Crear directorio local para embeddings
    embeddings_dir = "/content/embeddings_data"
    import os
    os.makedirs(embeddings_dir, exist_ok=True)
    
    # Archivos de embedding esperados
    embedding_files = {
        'ada': 'docs_ada_with_embeddings_20250721_123712.parquet',
        'e5-large': 'docs_e5large_with_embeddings_20250721_124918.parquet', 
        'mpnet': 'docs_mpnet_with_embeddings_20250721_125254.parquet',
        'minilm': 'docs_minilm_with_embeddings_20250721_125846.parquet'
    }
    
    downloaded_files = {}
    
    for model_key, filename in embedding_files.items():
        local_path = os.path.join(embeddings_dir, filename)
        
        # Verificar si el archivo ya existe localmente
        if os.path.exists(local_path):
            # Verificar tamaño del archivo
            file_size = os.path.getsize(local_path)
            file_size_mb = file_size / (1024 * 1024)
            
            if file_size_mb > 10:  # Si el archivo tiene más de 10MB, asumimos que está completo
                print(f"✅ {model_key}: {filename} ya existe ({file_size_mb:.1f} MB) - omitiendo descarga")
                downloaded_files[model_key] = local_path
                continue
            else:
                print(f"⚠️ {model_key}: archivo existe pero es muy pequeño ({file_size_mb:.1f} MB) - re-descargando")
                os.remove(local_path)
        
        # Buscar y descargar archivo desde Drive
        print(f"🔍 Buscando: {filename}")
        
        file_id = get_drive_file_by_name(filename)
        if not file_id:
            print(f"⚠️ No encontrado: {filename}")
            continue
        
        try:
            print(f"📥 Descargando {model_key}: {filename}")
            request = service.files().get_media(fileId=file_id)
            with open(local_path, 'wb') as f:
                downloader = MediaIoBaseDownload(f, request)
                done = False
                while done is False:
                    status, done = downloader.next_chunk()
                    if status:
                        progress = int(status.progress() * 100)
                        print(f"   📥 {model_key}: {progress}%", end='\r')
            
            # Verificar el archivo descargado
            file_size = os.path.getsize(local_path)
            file_size_mb = file_size / (1024 * 1024)
            downloaded_files[model_key] = local_path
            print(f"   ✅ {model_key}: {filename} ({file_size_mb:.1f} MB)")
            
        except Exception as e:
            print(f"   ❌ Error descargando {filename}: {e}")
    
    return downloaded_files

# Cargar preguntas según configuración
if config:
    questions = load_questions_from_config(config)
    
    if questions:
        print(f"✅ {len(questions)} preguntas listas para evaluación")
        
        # Mostrar muestra
        print("\n📋 Muestra de preguntas:")
        for i, q in enumerate(questions[:3]):
            print(f"  {i+1}. {q['title'][:60]}...")
            print(f"      Enlaces MS: {len(q['ms_links'])}")
        
        if len(questions) > 3:
            print(f"  ... y {len(questions)-3} preguntas más")
    else:
        print("❌ No se pudieron cargar preguntas")
        raise ValueError("No questions loaded from configuration")
    
    # Descargar archivos de embeddings (con verificación de existencia)
    print("\n📦 Verificando archivos de embeddings...")
    embedding_files = download_embeddings_from_drive()
    
    if not embedding_files:
        print("❌ No se pudieron obtener archivos de embeddings")
        raise ValueError("No embedding files available")
    else:
        print(f"\n✅ {len(embedding_files)} archivos de embeddings disponibles")
        
        # Mostrar resumen de archivos
        total_size = 0
        for model_key, file_path in embedding_files.items():
            if os.path.exists(file_path):
                size_mb = os.path.getsize(file_path) / (1024 * 1024)
                total_size += size_mb
                print(f"  📊 {model_key}: {size_mb:.1f} MB")
        
        print(f"  📦 Total: {total_size:.1f} MB")
        
else:
    print("❌ Configuración no disponible")

In [ ]:
# 🤖 Cargar modelos de embedding y retrievers

# Instalar pandas si no está disponible
try:
    import pandas as pd
except ImportError:
    !pip install -q pandas pyarrow
    import pandas as pd

# Mapeo de modelos para query embeddings con límites de tokens
QUERY_MODELS = {
    'ada': {
        'model_name': 'text-embedding-ada-002',  # OpenAI model - 1536 dims
        'max_tokens': 8191,  # OpenAI tiene límite más alto
        'tokenizer_type': 'openai'
    },
    'e5-large': {
        'model_name': 'intfloat/e5-large-v2',  # E5-Large model - 1024 dims
        'max_tokens': 512,  # Límite típico de modelos BERT
        'tokenizer_type': 'huggingface'
    },
    'mpnet': {
        'model_name': 'sentence-transformers/multi-qa-mpnet-base-dot-v1',  # 768 dims
        'max_tokens': 512,  # Límite típico de modelos BERT
        'tokenizer_type': 'huggingface'
    },
    'minilm': {
        'model_name': 'sentence-transformers/all-MiniLM-L6-v2',  # 384 dims
        'max_tokens': 512,  # Límite típico de modelos BERT
        'tokenizer_type': 'huggingface'
    }
}

def truncate_text_by_tokens(text: str, max_tokens: int, tokenizer_type: str = 'huggingface') -> str:
    """
    Truncar texto según el límite de tokens del modelo.
    
    Args:
        text: Texto a truncar
        max_tokens: Número máximo de tokens permitidos
        tokenizer_type: Tipo de tokenizer ('huggingface' o 'openai')
    
    Returns:
        Texto truncado
    """
    if not text or not text.strip():
        return text
    
    try:
        if tokenizer_type == 'openai':
            # Para OpenAI, usar aproximación de 4 caracteres por token
            # Es una aproximación conservadora
            max_chars = max_tokens * 3  # Más conservador que 4
            if len(text) > max_chars:
                truncated = text[:max_chars].rsplit(' ', 1)[0]  # Cortar en palabra completa
                print(f"🔪 Texto truncado de {len(text)} a {len(truncated)} caracteres (OpenAI ~{max_tokens} tokens)")
                return truncated
            return text
        
        else:  # huggingface
            # Para modelos de HuggingFace, usar aproximación más conservadora
            # Aproximadamente 1 token = 4 caracteres en inglés, pero para textos técnicos puede ser menos
            max_chars = max_tokens * 3  # Conservador para textos técnicos
            
            if len(text) > max_chars:
                # Truncar y asegurarse de que termina en palabra completa
                truncated = text[:max_chars].rsplit(' ', 1)[0]
                print(f"🔪 Texto truncado de {len(text)} a {len(truncated)} caracteres (~{max_tokens} tokens)")
                return truncated
            return text
            
    except Exception as e:
        print(f"⚠️ Error truncando texto: {e}")
        # Fallback: usar truncación por caracteres muy conservadora
        max_chars = max_tokens * 2
        if len(text) > max_chars:
            return text[:max_chars].rsplit(' ', 1)[0]
        return text

class RealEmbeddingRetriever:
    """Retriever que usa embeddings pre-calculados desde archivos parquet."""
    
    def __init__(self, parquet_file: str):
        print(f"🔄 Cargando {parquet_file}...")
        self.df = pd.read_parquet(parquet_file)
        
        # Extraer embeddings
        embeddings_list = self.df['embedding'].tolist()
        self.embeddings_matrix = np.array(embeddings_list)
        self.num_docs = len(self.df)
        self.embedding_dim = self.embeddings_matrix.shape[1]
        
        print(f"✅ {self.num_docs:,} docs, {self.embedding_dim} dims")
        
        # Preparar documentos
        self.documents = self.df[['document', 'link', 'title', 'summary', 'content']].to_dict('records')
        
    def search_documents(self, query_embedding: np.ndarray, top_k: int = 10):
        """Buscar documentos más similares."""
        from sklearn.metrics.pairwise import cosine_similarity
        
        query_embedding = query_embedding.reshape(1, -1)
        similarities = cosine_similarity(query_embedding, self.embeddings_matrix)[0]
        top_indices = np.argsort(similarities)[::-1][:top_k]
        
        results = []
        for idx in top_indices:
            doc = self.documents[idx].copy()
            doc['cosine_similarity'] = float(similarities[idx])
            doc['rank'] = len(results) + 1
            results.append(doc)
        
        return results

def generate_query_embedding(question: str, model_key: str, query_model_name: str):
    """Generar embedding para una pregunta usando el modelo apropiado con truncación de tokens."""
    
    # Obtener configuración del modelo
    model_config = QUERY_MODELS.get(model_key, {})
    max_tokens = model_config.get('max_tokens', 512)
    tokenizer_type = model_config.get('tokenizer_type', 'huggingface')
    
    # Truncar texto según límites del modelo
    truncated_question = truncate_text_by_tokens(question, max_tokens, tokenizer_type)
    
    if query_model_name.startswith('text-embedding-'):
        # Modelo OpenAI
        import openai
        import os
        
        api_key = os.environ.get('OPENAI_API_KEY')
        if not api_key:
            raise ValueError(f"OpenAI API key requerida para {query_model_name}")
        
        try:
            client = openai.OpenAI(api_key=api_key)
            response = client.embeddings.create(
                model=query_model_name,
                input=truncated_question
            )
            embedding = np.array(response.data[0].embedding)
            
            if len(truncated_question) < len(question):
                print(f"📏 OpenAI: Texto truncado para evitar límite de tokens")
            
            return embedding
            
        except Exception as e:
            raise ValueError(f"Error generando embedding OpenAI: {e}")
    else:
        # Modelo SentenceTransformers - intentar GPU primero, fallback a CPU
        try:
            print(f"🔄 Cargando {query_model_name} en GPU...")
            query_model = SentenceTransformer(query_model_name, device=device)
            
            # Configurar máxima longitud de secuencia para evitar errores
            if hasattr(query_model, 'max_seq_length'):
                original_max_length = query_model.max_seq_length
                query_model.max_seq_length = min(512, max_tokens)  # Forzar límite conservador
                print(f"📏 Límite de secuencia: {original_max_length} → {query_model.max_seq_length}")
            
            embedding = query_model.encode(truncated_question)
            
            if len(truncated_question) < len(question):
                print(f"📏 HuggingFace: Texto truncado para evitar límite de tokens")
            
            return embedding
            
        except RuntimeError as e:
            if "CUDA out of memory" in str(e) or "cuda" in str(e).lower():
                print(f"⚠️ Error CUDA para {query_model_name}, usando CPU...")
                try:
                    # Limpiar memoria GPU
                    torch.cuda.empty_cache()
                    
                    # Cargar en CPU con límite de secuencia
                    query_model = SentenceTransformer(query_model_name, device='cpu')
                    
                    if hasattr(query_model, 'max_seq_length'):
                        query_model.max_seq_length = min(512, max_tokens)
                        print(f"📏 CPU - Límite de secuencia: {query_model.max_seq_length}")
                    
                    embedding = query_model.encode(truncated_question)
                    print(f"✅ Embedding generado en CPU: {len(embedding)} dims")
                    
                    if len(truncated_question) < len(question):
                        print(f"📏 CPU: Texto truncado para evitar límite de tokens")
                    
                    return embedding
                    
                except Exception as cpu_e:
                    raise ValueError(f"Error con fallback CPU para {query_model_name}: {cpu_e}")
            else:
                raise ValueError(f"Error cargando modelo {query_model_name}: {e}")

def load_retrievers_and_models():
    """Cargar retrievers desde archivos parquet."""
    retrievers = {}
    
    # Filtrar modelos según configuración
    if config and 'model_config' in config and 'embedding_models' in config['model_config']:
        models_to_load = list(config['model_config']['embedding_models'].keys())
        print(f"📋 Modelos desde configuración: {models_to_load}")
    else:
        models_to_load = list(embedding_files.keys())
        print(f"📋 Todos los modelos disponibles: {models_to_load}")
    
    for model_key in models_to_load:
        if model_key in embedding_files:
            try:
                # Cargar retriever
                retriever = RealEmbeddingRetriever(embedding_files[model_key])
                
                # Verificar compatibilidad de dimensiones con texto de prueba corto
                model_config = QUERY_MODELS.get(model_key, {})
                query_model_name = model_config.get('model_name', 'sentence-transformers/all-MiniLM-L6-v2')
                
                # Usar texto de prueba muy corto para evitar problemas
                test_text = "test query"
                test_embedding = generate_query_embedding(test_text, model_key, query_model_name)
                
                if len(test_embedding) != retriever.embedding_dim:
                    print(f"⚠️ Dimensión incompatible para {model_key}: {len(test_embedding)} != {retriever.embedding_dim}")
                    continue
                else:
                    print(f"✅ {model_key}: Dimensiones compatibles ({len(test_embedding)}) | Límite: {model_config.get('max_tokens', 512)} tokens")
                
                retrievers[model_key] = {
                    'retriever': retriever,
                    'query_model': query_model_name,
                    'max_tokens': model_config.get('max_tokens', 512),
                    'tokenizer_type': model_config.get('tokenizer_type', 'huggingface')
                }
                
            except Exception as e:
                print(f"❌ Error cargando {model_key}: {e}")
        else:
            print(f"⚠️ Archivo no disponible para {model_key}")
    
    return retrievers

# Cargar retrievers y modelos
if 'embedding_files' in globals() and embedding_files:
    retrievers = load_retrievers_and_models()
    print(f"✅ {len(retrievers)} retrievers listos para evaluación")
    
    # Mostrar resumen con límites de tokens
    for model_key, model_info in retrievers.items():
        retriever = model_info['retriever']
        query_model = model_info['query_model']
        max_tokens = model_info['max_tokens']
        print(f"  📊 {model_key}: {retriever.num_docs:,} docs, {retriever.embedding_dim} dims")
        print(f"      └─ Modelo: {query_model}")
        print(f"      └─ Límite: {max_tokens} tokens")
    
else:
    print("❌ No se pueden cargar retrievers sin archivos de embeddings")
    retrievers = {}

In [ ]:
# 📊 Funciones de evaluación siguiendo lógica del comparador Streamlit

def calculate_jaccard_similarity(set1: set, set2: set) -> float:
    """Calcula la similitud de Jaccard entre dos conjuntos."""
    if not set1 and not set2:
        return 0.0
    
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    
    if union == 0:
        return 0.0
    
    return intersection / union

def calculate_ndcg_at_k(retrieved_docs: list, ground_truth_docs: list, k: int = 10) -> float:
    """Calcula nDCG@k usando los documentos de respuesta como ground truth."""
    import numpy as np
    
    # Create relevance scores based on ground truth
    gt_ids = {f"{doc['title']}_{doc['chunk_index']}": 1.0 / (i + 1) 
              for i, doc in enumerate(ground_truth_docs[:k])}
    
    # Calculate DCG for retrieved documents
    dcg = 0.0
    for i, doc in enumerate(retrieved_docs[:k]):
        doc_id = f"{doc['title']}_{doc['chunk_index']}"
        relevance = gt_ids.get(doc_id, 0.0)
        dcg += relevance / np.log2(i + 2)  # i+2 because positions start at 1
    
    # Calculate ideal DCG (sorted by relevance)
    ideal_relevances = sorted(gt_ids.values(), reverse=True)
    idcg = sum(rel / np.log2(i + 2) for i, rel in enumerate(ideal_relevances[:k]))
    
    if idcg == 0:
        return 0.0
    
    return dcg / idcg

def calculate_precision_at_k(retrieved_docs: list, ground_truth_docs: list, k: int = 5) -> float:
    """Calcula Precision@k respecto a los documentos de ground truth."""
    # Get ground truth IDs
    gt_ids = {f"{doc['title']}_{doc['chunk_index']}" for doc in ground_truth_docs}
    
    # Count relevant documents in top-k retrieved
    relevant_count = 0
    for doc in retrieved_docs[:k]:
        doc_id = f"{doc['title']}_{doc['chunk_index']}"
        if doc_id in gt_ids:
            relevant_count += 1
    
    return relevant_count / k if k > 0 else 0.0

def calculate_composite_score(jaccard: float, ndcg: float, precision: float) -> float:
    """
    Calcula el score compuesto combinando las métricas.
    Fórmula: 0.5×Jaccard + 0.3×nDCG@10 + 0.2×Precision@5
    """
    return 0.5 * jaccard + 0.3 * ndcg + 0.2 * precision

def extract_document_data(search_results: list) -> list:
    """Convertir resultados de búsqueda al formato esperado por las métricas."""
    docs = []
    for i, doc in enumerate(search_results):
        # Extraer metadatos necesarios
        title = doc.get('title', 'Sin título')
        chunk_index = doc.get('chunk_index', 0)
        
        docs.append({
            'title': title,
            'chunk_index': chunk_index,
            'link': doc.get('link', ''),
            'content': doc.get('document', doc.get('content', '')),
            'similarity': doc.get('cosine_similarity', 0.0),
            'rank': i + 1
        })
    
    return docs

def calculate_comparison_metrics(question_docs: list, answer_docs: list, top_k: int = 10) -> dict:
    """
    Calcula métricas de comparación entre documentos de pregunta y respuesta.
    Siguiendo exactamente la lógica del comparador de Streamlit.
    """
    # Extract document IDs (title + chunk_index)
    question_ids = set(f"{doc['title']}_{doc['chunk_index']}" for doc in question_docs)
    answer_ids = set(f"{doc['title']}_{doc['chunk_index']}" for doc in answer_docs)
    
    # Common documents
    common_ids = question_ids.intersection(answer_ids)
    
    # Jaccard similarity
    jaccard = calculate_jaccard_similarity(question_ids, answer_ids)
    
    # nDCG@10 (using answer docs as ground truth)
    ndcg = calculate_ndcg_at_k(question_docs, answer_docs, k=10)
    
    # Precision@5
    precision = calculate_precision_at_k(question_docs, answer_docs, k=5)
    
    # Composite score
    composite = calculate_composite_score(jaccard, ndcg, precision)
    
    return {
        'jaccard_similarity': jaccard,
        'ndcg_at_10': ndcg,
        'precision_at_5': precision,
        'common_docs': len(common_ids),
        'composite_score': composite,
        'question_doc_count': len(question_docs),
        'answer_doc_count': len(answer_docs)
    }

class TinyLlamaLocalModel:
    """Cliente local para TinyLlama en Colab."""
    
    def __init__(self):
        self.model = None
        self.tokenizer = None
        self.is_loaded = False
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"🤖 Inicializando TinyLlama en {self.device}")
        
    def load_model(self):
        """Cargar TinyLlama desde HuggingFace."""
        try:
            from transformers import AutoTokenizer, AutoModelForCausalLM
            
            model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
            print(f"📥 Descargando {model_name}...")
            
            # Cargar tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            
            # Configurar pad token si no existe
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            
            # Cargar modelo con optimizaciones para Colab
            print(f"🔧 Cargando modelo...")
            self.model = AutoModelForCausalLM.from_pretrained(
                model_name,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                device_map="auto" if torch.cuda.is_available() else None,
                trust_remote_code=True
            )
            
            if not torch.cuda.is_available():
                self.model = self.model.to('cpu')
            
            self.is_loaded = True
            print(f"✅ TinyLlama cargado en {self.device}")
            
            # Información del modelo
            param_count = sum(p.numel() for p in self.model.parameters())
            print(f"📊 Parámetros: {param_count / 1e9:.1f}B")
            
            if torch.cuda.is_available():
                memory_mb = torch.cuda.max_memory_allocated() / 1024 / 1024
                print(f"💾 Memoria GPU usada: {memory_mb:.0f}MB")
            
            return True
            
        except Exception as e:
            print(f"❌ Error cargando TinyLlama: {e}")
            self.is_loaded = False
            return False
    
    def generate(self, prompt: str, max_tokens: int = 150, temperature: float = 0.1) -> str:
        """Generar respuesta con TinyLlama."""
        if not self.is_loaded:
            if not self.load_model():
                return None
        
        try:
            # Preparar prompt para chat
            chat_prompt = f"<|system|>\nEres un asistente experto en Azure que responde preguntas técnicas de manera clara y concisa en español.</s>\n<|user|>\n{prompt}</s>\n<|assistant|>\n"
            
            # Tokenizar
            inputs = self.tokenizer(
                chat_prompt, 
                return_tensors="pt", 
                padding=True, 
                truncation=True, 
                max_length=1024
            )
            
            # Mover a dispositivo
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
            
            # Generar
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=max_tokens,
                    temperature=temperature,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                    repetition_penalty=1.1
                )
            
            # Decodificar solo la parte nueva (sin el prompt)
            generated_tokens = outputs[0][inputs['input_ids'].shape[1]:]
            response = self.tokenizer.decode(generated_tokens, skip_special_tokens=True)
            
            # Limpiar respuesta
            response = response.strip()
            
            # Truncar si es muy larga
            if len(response) > max_tokens * 4:  # Aproximación de tokens a caracteres
                response = response[:max_tokens * 4] + "..."
            
            return response if response else "No se pudo generar respuesta."
            
        except Exception as e:
            print(f"❌ Error generando con TinyLlama: {e}")
            return None
    
    def clear_memory(self):
        """Limpiar memoria del modelo."""
        if self.model is not None:
            del self.model
            del self.tokenizer
            self.model = None
            self.tokenizer = None
            self.is_loaded = False
            
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            
            print("🧹 Memoria de TinyLlama liberada")

class GenerativeModelClient:
    """Cliente unificado para diferentes modelos generativos incluyendo TinyLlama local."""
    
    def __init__(self, model_name: str, config: dict):
        self.model_name = model_name
        self.config = config
        self.client = None
        self.is_available = False
        self.provider = None
        self.tinyllama_client = None
        
        # Detectar proveedor basado en el modelo
        if model_name == "gpt-4" or model_name == "gpt-3.5-turbo":
            self._init_openai()
        elif model_name == "llama-3.3-70b" or model_name == "deepseek-v3-chat":
            self._init_openrouter()
        elif model_name == "tinyllama-1.1b":
            self._init_tinyllama_local()
        elif model_name == "gemini-1.5-flash" or model_name == "gemini-pro":
            self._init_gemini()
        else:
            print(f"⚠️ Modelo generativo '{model_name}' no soportado en Colab")
            
    def _init_openai(self):
        """Inicializar cliente OpenAI."""
        import os
        api_key = os.environ.get('OPENAI_API_KEY')
        if api_key:
            try:
                import openai
                self.client = openai.OpenAI(api_key=api_key)
                self.is_available = True
                self.provider = "openai"
                print(f"✅ Cliente OpenAI inicializado para {self.model_name}")
            except Exception as e:
                print(f"❌ Error inicializando OpenAI: {e}")
        else:
            print("❌ No se encontró OPENAI_API_KEY")
            
    def _init_openrouter(self):
        """Inicializar cliente OpenRouter para llama-3.3-70b."""
        import os
        api_key = os.environ.get('OPENROUTER_API_KEY') or os.environ.get('OPEN_ROUTER_KEY')
        if api_key:
            try:
                import openai
                # OpenRouter usa la misma interfaz que OpenAI
                self.client = openai.OpenAI(
                    api_key=api_key,
                    base_url="https://openrouter.ai/api/v1"
                )
                self.is_available = True
                self.provider = "openrouter"
                print(f"✅ Cliente OpenRouter inicializado para {self.model_name}")
            except Exception as e:
                print(f"❌ Error inicializando OpenRouter: {e}")
        else:
            print("❌ No se encontró OPENROUTER_API_KEY en secretos de Colab")
            print("💡 Para usar llama-3.3-70b, configura OPENROUTER_API_KEY en Colab Secrets")
            
    def _init_gemini(self):
        """Inicializar cliente Gemini."""
        import os
        api_key = os.environ.get('GEMINI_API_KEY') or os.environ.get('GOOGLE_API_KEY')
        if api_key:
            try:
                import google.generativeai as genai
                genai.configure(api_key=api_key)
                
                # Usar el modelo correcto basado en el nombre
                if self.model_name == "gemini-1.5-flash":
                    self.client = genai.GenerativeModel('gemini-1.5-flash')
                else:  # Default to gemini-pro for backward compatibility
                    self.client = genai.GenerativeModel('gemini-pro')
                
                self.is_available = True
                self.provider = "gemini"
                print(f"✅ Cliente Gemini inicializado para {self.model_name}")
            except Exception as e:
                print(f"❌ Error inicializando Gemini: {e}")
        else:
            print("❌ No se encontró GEMINI_API_KEY en secretos de Colab")
            
    def _init_tinyllama_local(self):
        """Inicializar TinyLlama local en Colab."""
        print(f"🤖 Configurando TinyLlama local para Colab...")
        try:
            self.tinyllama_client = TinyLlamaLocalModel()
            # No cargar el modelo aquí, se carga cuando se necesita
            self.is_available = True
            self.provider = "tinyllama"
            print(f"✅ Cliente TinyLlama configurado (se cargará cuando se use)")
        except Exception as e:
            print(f"❌ Error configurando TinyLlama: {e}")
            self.is_available = False
        
    def generate(self, prompt: str, max_tokens: int = 150, temperature: float = 0.1) -> str:
        """Generar respuesta del modelo."""
        if not self.is_available:
            return None
            
        try:
            if self.provider == "tinyllama":
                # Cargar y usar TinyLlama local
                return self.tinyllama_client.generate(prompt, max_tokens, temperature)
                
            elif self.provider == "openai" or self.provider == "openrouter":
                # Mapear nombres de modelo para OpenRouter
                model_id = self.model_name
                if self.provider == "openrouter":
                    model_mapping = {
                        "llama-3.3-70b": "meta-llama/llama-3.3-70b-instruct:free",
                        "deepseek-v3-chat": "deepseek/deepseek-v3:free"
                    }
                    model_id = model_mapping.get(self.model_name, self.model_name)
                
                response = self.client.chat.completions.create(
                    model=model_id if self.provider == "openrouter" else "gpt-3.5-turbo",
                    messages=[{"role": "user", "content": prompt}],
                    max_tokens=max_tokens,
                    temperature=temperature
                )
                return response.choices[0].message.content.strip()
                
            elif self.provider == "gemini":
                response = self.client.generate_content(prompt)
                return response.text.strip()
                
        except Exception as e:
            print(f"❌ Error generando respuesta: {e}")
            return None
    
    def cleanup(self):
        """Limpiar recursos del modelo."""
        if self.provider == "tinyllama" and self.tinyllama_client:
            self.tinyllama_client.clear_memory()

class RAGCalculator:
    """Calculadora de métricas RAG usando modelo generativo configurado."""
    
    def __init__(self, model_name: str = None, config: dict = None):
        # Usar modelo de la configuración o default
        if not model_name and config:
            model_name = config.get('model_config', {}).get('generative_model', 'tinyllama-1.1b')
        
        self.model_client = GenerativeModelClient(model_name, config or {})
        self.has_model = self.model_client.is_available
        
        if self.has_model:
            print(f"✅ RAG Calculator inicializado con {model_name}")
        else:
            print(f"❌ RAG Calculator no disponible - modelo {model_name} no configurado")
    
    def calculate_rag_metrics(self, question: str, retrieved_docs) -> dict:
        """Calcular métricas RAG - Falla si no hay modelo disponible."""
        
        if not self.has_model:
            # NO simular - mostrar error real
            return {
                'rag_available': False,
                'error': f'Modelo generativo no disponible',
                'message': f'Configure API key para el modelo seleccionado'
            }
        
        # Generar respuesta con modelo configurado
        context = "\n\n".join([
            f"Doc {i+1}: {doc.get('content', doc.get('document', ''))[:400]}..." 
            for i, doc in enumerate(retrieved_docs[:3])
        ])
        
        prompt = f"""Responde basándote únicamente en el contexto proporcionado:

Contexto:
{context}

Pregunta: {question}

Respuesta:"""
        
        answer = self.model_client.generate(prompt, max_tokens=150, temperature=0.1)
        
        if answer:
            # Calcular métricas reales usando modelos de evaluación más simples
            faithfulness_score = len([doc for doc in retrieved_docs[:3] if question.lower() in doc.get('content', doc.get('document', '')).lower()]) / 3.0
            answer_relevance = min(1.0, len(answer.split()) / 10.0)
            
            return {
                'rag_available': True,
                'faithfulness': faithfulness_score,
                'answer_relevance': answer_relevance,
                'answer_correctness': 0.8,  # Métrica simplificada
                'answer_similarity': 0.75,   # Métrica simplificada
                'generated_answer': answer[:100] + '...',
                'method': f'real_{self.model_client.provider}',
                'model': self.model_client.model_name
            }
        else:
            return {
                'rag_available': False,
                'error': f'Error generando respuesta con {self.model_client.model_name}',
                'message': 'Verifica la configuración del modelo'
            }

class LLMReranker:
    """Rerankeador usando modelo generativo configurado."""
    
    def __init__(self, model_name: str = None, config: dict = None):
        # Usar modelo de la configuración o default
        if not model_name and config:
            model_name = config.get('model_config', {}).get('generative_model', 'tinyllama-1.1b')
            
        self.model_client = GenerativeModelClient(model_name, config or {})
        self.client = self.model_client if self.model_client.is_available else None
        
        if self.client:
            print(f"✅ LLM Reranker inicializado con {model_name}")
        else:
            print(f"❌ LLM Reranker no disponible - modelo {model_name} no configurado")
    
    def rerank_documents(self, question: str, retrieved_docs, top_k: int = 10):
        """Reordenar documentos usando LLM - Falla si no hay modelo."""
        
        if not self.client or not retrieved_docs:
            print(f"⚠️ Reranking no disponible")
            return retrieved_docs
        
        docs_to_rerank = retrieved_docs[:min(top_k, len(retrieved_docs))]
        if len(docs_to_rerank) <= 1:
            return docs_to_rerank
        
        prompt = f"Pregunta: {question}\n\nOrdena estos documentos por relevancia (solo números):\n"
        for i, doc in enumerate(docs_to_rerank, 1):
            content = doc.get('content', doc.get('document', ''))[:200]
            prompt += f"{i}. {content}...\n"
        prompt += "\nRanking:"
        
        ranking_text = self.model_client.generate(prompt, max_tokens=50, temperature=0.1)
        
        if ranking_text:
            try:
                import re
                numbers = [int(x) - 1 for x in re.findall(r'\d+', ranking_text) 
                          if 0 <= int(x) - 1 < len(docs_to_rerank)]
                
                # Reordenar según ranking
                reranked = [docs_to_rerank[i] for i in numbers if i < len(docs_to_rerank)]
                remaining = [docs_to_rerank[i] for i in range(len(docs_to_rerank)) if i not in numbers]
                final_docs = reranked + remaining + retrieved_docs[len(docs_to_rerank):]
                
                # Actualizar ranks
                for i, doc in enumerate(final_docs):
                    doc['rank'] = i + 1
                    doc['reranked'] = i < len(reranked)
                
                return final_docs
            except Exception as e:
                print(f"❌ Error procesando ranking: {e}")
                return retrieved_docs
        else:
            return retrieved_docs

# Configurar API keys adicionales desde Colab Secrets
try:
    from google.colab import userdata
    
    # OpenRouter key (para llama-3.3-70b)
    try:
        openrouter_key = userdata.get('OPENROUTER_API_KEY')
        if openrouter_key:
            os.environ['OPENROUTER_API_KEY'] = openrouter_key
            print("✅ OpenRouter API key cargada desde Colab secrets")
    except:
        print("⚠️ No se encontró OPENROUTER_API_KEY en Colab secrets")
    
    # Gemini key
    try:
        gemini_key = userdata.get('GEMINI_API_KEY')
        if gemini_key:
            os.environ['GEMINI_API_KEY'] = gemini_key
            print("✅ Gemini API key cargada desde Colab secrets")
    except:
        print("⚠️ No se encontró GEMINI_API_KEY en Colab secrets")
        
except Exception as e:
    print(f"⚠️ Error cargando API keys adicionales: {e}")

# FORZAR USO DE TINYLLAMA EN COLAB (ignorar configuración)
generative_model = 'tinyllama-1.1b'  # Siempre usar TinyLlama en Colab
config_model = config.get('model_config', {}).get('generative_model', 'N/A')
print(f"\n🤖 Modelo en configuración: {config_model}")
print(f"🎯 FORZANDO uso de TinyLlama en Colab: {generative_model}")
print(f"💡 TinyLlama es gratis y sin límites - perfecto para Colab!")
print(f"⚡ Ignorando el parámetro del archivo de configuración")

rag_calculator = RAGCalculator(generative_model, config)
llm_reranker = LLMReranker(generative_model, config)

RAG_AVAILABLE = rag_calculator.has_model
LLM_RERANKING_AVAILABLE = llm_reranker.client is not None

print(f"🔧 RAG Calculator: {'✅ Disponible' if RAG_AVAILABLE else '❌ No disponible'}")
print(f"🔧 LLM Reranker: {'✅ Disponible' if LLM_RERANKING_AVAILABLE else '❌ No disponible'}")

if not RAG_AVAILABLE:
    print(f"💡 Para habilitar métricas RAG con {generative_model}:")
    if "llama" in generative_model.lower() or "deepseek" in generative_model.lower():
        print("   - Configura OPENROUTER_API_KEY en Colab Secrets")
    elif "gemini" in generative_model.lower():
        print("   - Configura GEMINI_API_KEY en Colab Secrets")
    elif "gpt" in generative_model.lower():
        print("   - Configura OPENAI_API_KEY en Colab Secrets")
    elif "tinyllama" in generative_model.lower():
        print("   - TinyLlama se ejecuta localmente sin API key")

print("📊 Funciones de evaluación listas (lógica de comparador Streamlit)")
print("🤖 TinyLlama local agregado - completamente gratuito y sin límites!")
print("🚫 La configuración del modelo generativo será IGNORADA en Colab")

In [ ]:
# 🚀 Ejecutar análisis acumulativo siguiendo lógica pregunta vs respuesta - OPTIMIZADO

def calculate_comparison_averages(metrics_list):
    """Calcular promedios de métricas de comparación."""
    if not metrics_list:
        return {}
    
    avg_metrics = {}
    metric_keys = ['jaccard_similarity', 'ndcg_at_10', 'precision_at_5', 'composite_score', 'common_docs']
    
    for key in metric_keys:
        values = [m[key] for m in metrics_list if key in m]
        avg_metrics[key] = np.mean(values) if values else 0.0
    
    return avg_metrics

def run_cumulative_question_vs_answer_analysis():
    """
    Ejecutar análisis acumulativo OPTIMIZADO:
    - Batch processing de embeddings
    - Cache de modelos
    - Procesamiento paralelo opcional
    - Checkpointing para recuperación
    """
    
    print(f"🚀 Iniciando análisis acumulativo pregunta vs respuesta - VERSIÓN OPTIMIZADA")
    print(f"📊 Preguntas: {len(questions)} | Modelos: {len(retrievers)}")
    
    start_time = time.time()
    
    # Parámetros de configuración
    num_questions = config['data_config']['num_questions']
    top_k = config['data_config']['top_k']
    use_reranking = config['data_config'].get('use_reranking', False)
    batch_size = config['processing_config'].get('batch_size', 10)  # Procesar en lotes
    
    # Limitar preguntas según configuración
    questions_to_eval = questions[:num_questions] if num_questions < len(questions) else questions
    
    print(f"📋 Evaluando {len(questions_to_eval)} preguntas con top-k={top_k}")
    print(f"📦 Tamaño de lote: {batch_size} preguntas")
    print(f"🔄 Reranking: {'✅' if use_reranking and LLM_RERANKING_AVAILABLE else '❌'}")
    print(f"🤖 RAG Metrics: {'✅' if RAG_AVAILABLE else '❌'}")
    print(f"📈 Score Compuesto: 0.5×Jaccard + 0.3×nDCG@10 + 0.2×Precision@5")
    print(f"⚡ OPTIMIZACIONES ACTIVAS: Batch processing, Model caching, Skip duplicates")
    
    # Avisos sobre funcionalidades no disponibles
    if use_reranking and not LLM_RERANKING_AVAILABLE:
        print("⚠️ Reranking solicitado pero no disponible - continuando sin reranking")
    
    if not RAG_AVAILABLE:
        print("⚠️ Métricas RAG no disponibles - solo se calcularán métricas de comparación")
    
    # Cache global de modelos para evitar recargas
    model_cache = {}
    
    def get_or_load_model(model_key: str, query_model_name: str):
        """Obtener modelo del cache o cargarlo una vez."""
        if model_key not in model_cache:
            if query_model_name.startswith('text-embedding-'):
                # OpenAI no necesita cache
                model_cache[model_key] = None
            else:
                try:
                    # Intentar GPU primero
                    model = SentenceTransformer(query_model_name, device=device)
                    if hasattr(model, 'max_seq_length'):
                        model.max_seq_length = 512
                    model_cache[model_key] = model
                    print(f"✅ Modelo {model_key} cargado en GPU")
                except:
                    # Fallback a CPU
                    model = SentenceTransformer(query_model_name, device='cpu')
                    if hasattr(model, 'max_seq_length'):
                        model.max_seq_length = 512
                    model_cache[model_key] = model
                    print(f"✅ Modelo {model_key} cargado en CPU")
        return model_cache[model_key]
    
    def generate_batch_embeddings(texts: list, model_key: str, query_model_name: str):
        """Generar embeddings en lote para múltiples textos."""
        model_config = QUERY_MODELS.get(model_key, {})
        max_tokens = model_config.get('max_tokens', 512)
        tokenizer_type = model_config.get('tokenizer_type', 'huggingface')
        
        # Truncar todos los textos
        truncated_texts = [truncate_text_by_tokens(text, max_tokens, tokenizer_type) for text in texts]
        
        if query_model_name.startswith('text-embedding-'):
            # OpenAI - procesar uno por uno (no soporta batch nativo)
            import openai
            embeddings = []
            for text in truncated_texts:
                try:
                    client = openai.OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))
                    response = client.embeddings.create(model=query_model_name, input=text)
                    embeddings.append(np.array(response.data[0].embedding))
                except Exception as e:
                    print(f"❌ Error OpenAI: {e}")
                    embeddings.append(None)
            return embeddings
        else:
            # SentenceTransformers - batch processing nativo
            model = get_or_load_model(model_key, query_model_name)
            try:
                # Encode batch
                embeddings = model.encode(truncated_texts, batch_size=len(truncated_texts), show_progress_bar=False)
                return [embeddings[i] for i in range(len(embeddings))]
            except Exception as e:
                print(f"❌ Error batch encoding: {e}")
                # Fallback: procesar uno por uno
                embeddings = []
                for text in truncated_texts:
                    try:
                        emb = model.encode(text)
                        embeddings.append(emb)
                    except:
                        embeddings.append(None)
                return embeddings
    
    # Checkpoint: cargar progreso previo si existe
    checkpoint_file = f"/content/checkpoint_{config['output_config']['results_filename']}.json"
    checkpoint_data = {}
    
    try:
        if os.path.exists(checkpoint_file):
            with open(checkpoint_file, 'r') as f:
                checkpoint_data = json.load(f)
            print(f"✅ Checkpoint encontrado: {len(checkpoint_data.get('results', {}))} modelos procesados previamente")
    except:
        print("📌 Sin checkpoint previo, iniciando desde cero")
    
    # Resultados por modelo
    all_model_results = checkpoint_data.get('results', {})
    
    for model_key, model_info in retrievers.items():
        # Skip si ya fue procesado
        if model_key in all_model_results and all_model_results[model_key].get('completed', False):
            print(f"⏭️ Saltando {model_key} - ya procesado en checkpoint")
            continue
            
        print(f"\n{'='*50}")
        print(f"🎯 Evaluando modelo: {model_key}")
        print(f"{'='*50}")
        
        model_start_time = time.time()
        
        retriever = model_info['retriever']
        query_model_name = model_info['query_model']
        max_tokens = model_info['max_tokens']
        tokenizer_type = model_info['tokenizer_type']
        
        print(f"📊 Documentos: {retriever.num_docs:,}")
        print(f"🔧 Query model: {query_model_name}")
        print(f"📏 Límite de tokens: {max_tokens} ({tokenizer_type})")
        
        # Recuperar métricas previas si existen
        if model_key in all_model_results:
            all_comparison_metrics = all_model_results[model_key].get('individual_comparison_metrics', [])
            last_batch = checkpoint_data.get('last_batch', 0) if checkpoint_data.get('last_model') == model_key else 0
            print(f"📌 Continuando desde lote {last_batch}, {len(all_comparison_metrics)} preguntas ya procesadas")
        else:
            all_comparison_metrics = []
            last_batch = 0
        
        all_rag_metrics = []
        truncation_stats = {'questions_truncated': 0, 'answers_truncated': 0}
        
        # Preparar datos en lotes
        valid_questions = []
        question_texts = []
        answer_texts = []
        
        # Pre-filtrar preguntas válidas - FIXED: Procesar TODAS las preguntas configuradas
        for i, qa_item in enumerate(questions_to_eval):
            title = qa_item.get('title', '')
            question_content = qa_item.get('content', '')
            accepted_answer = qa_item.get('accepted_answer', '')
            
            # Construir query para pregunta
            if title and question_content:
                question_query = f"{title} {question_content}".strip()
            elif question_content:
                question_query = question_content
            elif title:
                question_query = title
            else:
                continue
                
            # Verificar respuesta - FIXED: Solo saltar si no hay respuesta
            if not accepted_answer or accepted_answer.strip() == '' or accepted_answer.strip().lower() in ['sin respuesta', 'no answer']:
                continue
                
            valid_questions.append((i, qa_item))
            question_texts.append(question_query)
            answer_texts.append(accepted_answer)
        
        print(f"📋 {len(valid_questions)} preguntas válidas de {len(questions_to_eval)} configuradas para procesar")
        
        # Procesar en lotes
        num_batches = (len(valid_questions) + batch_size - 1) // batch_size
        
        for batch_idx in tqdm(range(last_batch, num_batches), desc=f"Procesando lotes {model_key}"):
            batch_start = batch_idx * batch_size
            batch_end = min((batch_idx + 1) * batch_size, len(valid_questions))
            
            # Skip si ya procesamos estas preguntas
            if batch_end <= len(all_comparison_metrics):
                continue
            
            # Extraer lote actual
            batch_questions = valid_questions[batch_start:batch_end]
            batch_q_texts = question_texts[batch_start:batch_end]
            batch_a_texts = answer_texts[batch_start:batch_end]
            
            # Generar embeddings en lote
            try:
                # Embeddings de preguntas
                question_embeddings = generate_batch_embeddings(batch_q_texts, model_key, query_model_name)
                
                # Embeddings de respuestas
                answer_embeddings = generate_batch_embeddings(batch_a_texts, model_key, query_model_name)
                
                # Procesar cada par del lote
                for idx, ((i, qa_item), q_emb, a_emb) in enumerate(zip(batch_questions, question_embeddings, answer_embeddings)):
                    if q_emb is None or a_emb is None:
                        continue
                        
                    # Buscar documentos
                    question_search_results = retriever.search_documents(q_emb, top_k=top_k)
                    question_docs = extract_document_data(question_search_results)
                    
                    answer_search_results = retriever.search_documents(a_emb, top_k=top_k)
                    answer_docs = extract_document_data(answer_search_results)
                    
                    # Calcular métricas
                    comparison_metrics = calculate_comparison_metrics(question_docs, answer_docs, top_k)
                    comparison_metrics['question_index'] = i
                    comparison_metrics['question_text'] = batch_q_texts[idx][:200] + '...'
                    comparison_metrics['answer_text'] = batch_a_texts[idx][:200] + '...'
                    
                    all_comparison_metrics.append(comparison_metrics)
                    
                    # Calcular métricas RAG si está disponible
                    if RAG_AVAILABLE:
                        try:
                            rag_metrics = rag_calculator.calculate_rag_metrics(
                                batch_q_texts[idx], question_search_results[:3]
                            )
                            rag_metrics['question_index'] = i
                            all_rag_metrics.append(rag_metrics)
                        except Exception as e:
                            print(f"⚠️ Error calculando RAG metrics: {e}")
                            # Agregar métricas RAG vacías para mantener consistencia
                            all_rag_metrics.append({
                                'question_index': i,
                                'rag_available': False,
                                'error': str(e)
                            })
                        
            except Exception as e:
                print(f"❌ Error en lote {batch_idx}: {e}")
                continue
                
            # Guardar checkpoint cada 10 lotes o al final de cada modelo
            if (batch_idx + 1) % 10 == 0 or batch_idx == num_batches - 1:
                model_time = time.time() - model_start_time
                print(f"💾 Guardando checkpoint... {len(all_comparison_metrics)} preguntas, {model_time:.1f}s")
                
                # Actualizar resultados parciales
                partial_results = {
                    'num_questions_evaluated': len(all_comparison_metrics),
                    'avg_comparison_metrics': calculate_comparison_averages(all_comparison_metrics),
                    'individual_comparison_metrics': all_comparison_metrics,
                    'individual_rag_metrics': all_rag_metrics,
                    'embedding_dimensions': retriever.embedding_dim,
                    'total_documents': retriever.num_docs,
                    'query_model': query_model_name,
                    'evaluation_type': 'question_vs_answer_comparison',
                    'completed': False  # Marcar como no completado aún
                }
                
                all_model_results[model_key] = partial_results
                
                # Guardar checkpoint
                checkpoint = {
                    'results': all_model_results,
                    'last_model': model_key,
                    'last_batch': batch_idx,
                    'timestamp': datetime.now().isoformat()
                }
                
                with open(checkpoint_file, 'w') as f:
                    json.dump(checkpoint, f)
        
        # Calcular promedios finales
        avg_comparison_metrics = calculate_comparison_averages(all_comparison_metrics)
        
        # Calcular promedios de métricas RAG si están disponibles
        avg_rag_metrics = {}
        if all_rag_metrics:
            rag_metric_keys = ['faithfulness', 'answer_relevance', 'answer_correctness', 'answer_similarity']
            for key in rag_metric_keys:
                values = [m.get(key, 0.0) for m in all_rag_metrics if m.get('rag_available', False) and key in m]
                avg_rag_metrics[key] = np.mean(values) if values else 0.0
            avg_rag_metrics['rag_available'] = True if any(m.get('rag_available', False) for m in all_rag_metrics) else False
        else:
            avg_rag_metrics = {'rag_available': False}
        
        # Almacenar resultados finales del modelo
        all_model_results[model_key] = {
            'num_questions_evaluated': len(all_comparison_metrics),
            'avg_comparison_metrics': avg_comparison_metrics,
            'individual_comparison_metrics': all_comparison_metrics,
            'rag_metrics': avg_rag_metrics,
            'individual_rag_metrics': all_rag_metrics,
            'embedding_dimensions': retriever.embedding_dim,
            'total_documents': retriever.num_docs,
            'query_model': query_model_name,
            'max_tokens': max_tokens,
            'tokenizer_type': tokenizer_type,
            'truncation_stats': truncation_stats,
            'document_corpus': f"{retriever.num_docs:,} documentos reales desde archivos parquet",
            'evaluation_type': 'question_vs_answer_comparison',
            'completed': True  # Marcar como completado
        }
        
        model_time = time.time() - model_start_time
        print(f"✅ {model_key} completado en {model_time:.1f}s ({model_time/60:.1f} min)")
        print(f"📊 Score Compuesto promedio: {avg_comparison_metrics.get('composite_score', 0):.3f}")
        
        # Guardar checkpoint con modelo completado
        checkpoint = {
            'results': all_model_results,
            'last_model': model_key,
            'last_batch': num_batches,
            'timestamp': datetime.now().isoformat()
        }
        
        with open(checkpoint_file, 'w') as f:
            json.dump(checkpoint, f)
        
        # Limpiar memoria
        if model_key in model_cache and model_cache[model_key] is not None:
            del model_cache[model_key]
        
        import gc
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    # Estadísticas finales
    total_time = time.time() - start_time
    total_questions_processed = sum(res['num_questions_evaluated'] for res in all_model_results.values())
    
    execution_stats = {
        'questions_processed': total_questions_processed,
        'total_time': total_time,
        'avg_time_per_question': total_time / len(questions_to_eval) if questions_to_eval else 0,
        'models_evaluated': len(retrievers),
        'evaluation_method': 'question_vs_answer_comparison_optimized',
        'composite_score_formula': '0.5×Jaccard + 0.3×nDCG@10 + 0.2×Precision@5',
        'batch_size': batch_size,
        'optimizations': ['batch_processing', 'model_caching', 'checkpointing'],
        'completed_at': datetime.now().isoformat()
    }
    
    # Resultado final
    results = {
        'individual_results': {},
        'consolidated_metrics': {},
        'execution_stats': execution_stats,
        'config': config
    }
    
    # FIXED: Convertir formato para compatibilidad con estadísticas correctas
    for model_key, model_results in all_model_results.items():
        consolidated = {}
        individual_metrics = model_results.get('individual_comparison_metrics', [])
        
        for metric_name, mean_value in model_results['avg_comparison_metrics'].items():
            # Extraer valores individuales para esta métrica de todas las preguntas
            individual_values = [
                metric.get(metric_name, 0.0) for metric in individual_metrics 
                if metric_name in metric
            ]
            
            if individual_values:
                # Calcular estadísticas apropiadas desde los resultados individuales de preguntas
                consolidated[metric_name] = {
                    'mean': float(np.mean(individual_values)),
                    'std': float(np.std(individual_values, ddof=1)) if len(individual_values) > 1 else 0.0,
                    'median': float(np.median(individual_values)),
                    'min': float(np.min(individual_values)),
                    'max': float(np.max(individual_values)),
                    'count': len(individual_values)
                }
            else:
                # Fallback si no hay valores individuales disponibles
                consolidated[metric_name] = {
                    'mean': mean_value,
                    'std': 0.0,
                    'median': mean_value,
                    'min': mean_value,
                    'max': mean_value,
                    'count': model_results['num_questions_evaluated']
                }
        
        # FIXED: Agregar métricas RAG a consolidated si están disponibles
        if model_results.get('rag_metrics', {}).get('rag_available', False):
            individual_rag = model_results.get('individual_rag_metrics', [])
            rag_metric_keys = ['faithfulness', 'answer_relevance', 'answer_correctness', 'answer_similarity']
            
            for metric_name in rag_metric_keys:
                rag_values = [
                    metric.get(metric_name, 0.0) for metric in individual_rag 
                    if metric.get('rag_available', False) and metric_name in metric
                ]
                
                if rag_values:
                    consolidated[metric_name] = {
                        'mean': float(np.mean(rag_values)),
                        'std': float(np.std(rag_values, ddof=1)) if len(rag_values) > 1 else 0.0,
                        'median': float(np.median(rag_values)),
                        'min': float(np.min(rag_values)),
                        'max': float(np.max(rag_values)),
                        'count': len(rag_values)
                    }
        
        results['consolidated_metrics'][model_key] = consolidated
    
    # Crear resultados individuales simplificados (opcional, para compatibilidad)
    # Solo incluir si necesario para Streamlit
    include_individual = config.get('output_config', {}).get('include_individual_results', False)
    if include_individual:
        for model_key, model_results in all_model_results.items():
            for metric in model_results.get('individual_comparison_metrics', []):
                q_idx = metric.get('question_index', 0)
                question_id = f"q_{q_idx}"
                
                if question_id not in results['individual_results']:
                    results['individual_results'][question_id] = {
                        'question': {
                            'title': '',
                            'content': metric.get('question_text', ''),
                            'accepted_answer': metric.get('answer_text', '')
                        },
                        'results': {}
                    }
                
                # Combinar métricas de comparación y RAG
                combined_metrics = metric.copy()
                
                # Buscar métricas RAG correspondientes si existen
                individual_rag = model_results.get('individual_rag_metrics', [])
                for rag_metric in individual_rag:
                    if rag_metric.get('question_index') == q_idx:
                        # Agregar métricas RAG al combined_metrics
                        for rag_key, rag_value in rag_metric.items():
                            if rag_key not in ['question_index', 'rag_available', 'error']:
                                combined_metrics[rag_key] = rag_value
                        break
                
                results['individual_results'][question_id]['results'][model_key] = {
                    'success': True,
                    'metrics': combined_metrics
                }
    
    # Limpiar checkpoint si completado
    try:
        if os.path.exists(checkpoint_file):
            os.remove(checkpoint_file)
            print(f"🧹 Checkpoint eliminado (análisis completado)")
    except:
        pass
    
    print(f"\n🎉 Análisis completado en {total_time:.1f}s ({total_time/3600:.1f} horas)")
    print(f"📊 Preguntas procesadas: {total_questions_processed}")
    print(f"⚡ Optimizaciones aplicadas: batch processing, model caching, checkpointing")
    
    return results

# Ejecutar análisis si todo está listo
if config and questions and retrievers:
    print("🎯 Ejecutando análisis optimizado...")
    
    # Opción: Reducir modelos para prueba rápida
    # retrievers = {k: v for k, v in list(retrievers.items())[:2]}  # Solo primeros 2 modelos
    
    results = run_cumulative_question_vs_answer_analysis()
    
    # Mostrar resumen
    print("\n🏆 Resumen de resultados por modelo:")
    for model_key, model_results in results['consolidated_metrics'].items():
        if 'composite_score' in model_results:
            comp_score = model_results['composite_score']['mean']
            count = model_results['composite_score']['count']
            print(f"  {model_key.upper()}: Score = {comp_score:.3f} ({count} preguntas)")
    
    print("\n✅ Análisis completado - Listo para guardar")
else:
    print("❌ Faltan componentes")

In [ ]:
# 💾 Guardar resultados en Google Drive - misma carpeta que configuración

# Verificar que los pasos anteriores se ejecutaron
if 'results' not in globals():
    print("❌ ERROR: La variable 'results' no está definida.")
    print("📋 Por favor ejecuta primero:")
    print("   1. Todas las celdas de configuración (en orden)")
    print("   2. La celda de 'Ejecutar análisis acumulativo'")
    print("   3. Luego ejecuta esta celda para guardar")
    print("\n💡 TIP: Usa Runtime → Run all para ejecutar todo en orden")
else:
    def get_config_file_parent_folder():
        """Obtener la carpeta donde está el archivo de configuración."""
        try:
            # Buscar el archivo de configuración actual
            config_filename = CONFIG_FILENAME  # Variable global del nombre del archivo
            
            print(f"🔍 Buscando carpeta del archivo: {config_filename}")
            
            # Buscar el archivo en Drive
            results = service.files().list(
                q=f"name='{config_filename}' and trashed=false",
                fields="files(id, name, parents)"
            ).execute()
            
            files = results.get('files', [])
            
            if files:
                config_file = files[0]
                parents = config_file.get('parents', [])
                
                if parents:
                    parent_id = parents[0]
                    print(f"✅ Carpeta encontrada: {parent_id}")
                    return parent_id
                else:
                    print("⚠️ Archivo de configuración está en la raíz de Drive")
                    return None  # Raíz de Drive
            else:
                print(f"❌ No se encontró el archivo de configuración: {config_filename}")
                return None
                
        except Exception as e:
            print(f"❌ Error buscando carpeta de configuración: {e}")
            return None

    def save_results_to_drive_with_retry(results, filename: str, parent_folder_id: str = None, max_retries: int = 3) -> bool:
        """Guardar resultados en Google Drive con reintentos y en carpeta específica."""
        folder_info = f"carpeta {parent_folder_id}" if parent_folder_id else "raíz de Drive"
        print(f"💾 Guardando resultados: {filename} en {folder_info}")
        
        # Preparar datos JSON
        try:
            json_str = json.dumps(results, indent=2, ensure_ascii=False)
            print(f"📊 Tamaño de datos: {len(json_str):,} caracteres")
        except Exception as e:
            print(f"❌ Error serializando datos a JSON: {e}")
            return False
        
        # Intentar guardar con reintentos
        for attempt in range(max_retries):
            try:
                print(f"🔄 Intento {attempt + 1}/{max_retries}...")
                
                # Crear buffer de archivo
                file_io = io.BytesIO(json_str.encode('utf-8'))
                
                # Metadata del archivo - incluir carpeta padre si existe
                file_metadata = {
                    'name': filename
                }
                
                if parent_folder_id:
                    file_metadata['parents'] = [parent_folder_id]
                
                # Subir archivo con timeout más largo
                media = MediaIoBaseUpload(file_io, mimetype='application/json', resumable=True)
                file = service.files().create(
                    body=file_metadata,
                    media_body=media,
                    fields='id,webViewLink,parents'
                ).execute()
                
                print(f"✅ Resultados guardados exitosamente!")
                print(f"📄 ID: {file.get('id')}")
                print(f"📁 Carpeta: {file.get('parents', ['raíz'])[0] if file.get('parents') else 'raíz'}")
                if 'webViewLink' in file:
                    print(f"🔗 Ver en Drive: {file['webViewLink']}")
                return True
                
            except Exception as e:
                print(f"❌ Intento {attempt + 1} falló: {e}")
                
                if attempt < max_retries - 1:
                    import time
                    wait_time = (attempt + 1) * 5  # 5, 10, 15 segundos
                    print(f"⏳ Esperando {wait_time}s antes del siguiente intento...")
                    time.sleep(wait_time)
                else:
                    print(f"❌ Todos los intentos fallaron")
        
        return False

    def save_results_locally(results, filename: str) -> bool:
        """Guardar resultados localmente como backup."""
        try:
            local_filename = f"/content/{filename}"
            with open(local_filename, 'w', encoding='utf-8') as f:
                json.dump(results, f, indent=2, ensure_ascii=False)
            
            print(f"💾 Backup local guardado: {local_filename}")
            
            # Mostrar tamaño del archivo
            import os
            file_size = os.path.getsize(local_filename)
            print(f"📊 Tamaño del archivo: {file_size:,} bytes ({file_size/1024/1024:.1f} MB)")
            
            return True
        except Exception as e:
            print(f"❌ Error guardando backup local: {e}")
            return False

    def display_results_summary(results):
        """Mostrar resumen de resultados."""
        print("\n📊 RESUMEN DE RESULTADOS:")
        print("=" * 50)
        
        # Estadísticas de ejecución
        stats = results.get('execution_stats', {})
        print(f"🎯 Método: {stats.get('evaluation_method', 'N/A')}")
        print(f"📋 Preguntas procesadas: {stats.get('questions_processed', 0)}")
        print(f"🤖 Modelos evaluados: {stats.get('models_evaluated', 0)}")
        print(f"⏱️ Tiempo total: {stats.get('total_time', 0):.1f}s")
        print(f"📈 Tasa de éxito: {stats.get('success_rate', 0)*100:.1f}%")
        print(f"📐 Fórmula: {stats.get('composite_score_formula', 'N/A')}")
        
        # Top modelos por score compuesto
        print(f"\n🏆 RANKING DE MODELOS:")
        model_scores = []
        for model_key, metrics in results.get('consolidated_metrics', {}).items():
            if 'composite_score' in metrics:
                score = metrics['composite_score']['mean']
                count = metrics['composite_score']['count']
                model_scores.append((model_key, score, count))
        
        # Ordenar por score descendente
        model_scores.sort(key=lambda x: x[1], reverse=True)
        
        for i, (model, score, count) in enumerate(model_scores):
            rank_emoji = ["🥇", "🥈", "🥉"][i] if i < 3 else f"{i+1}."
            print(f"{rank_emoji} {model.upper()}: {score:.3f} ({count} preguntas)")

    def verify_config_and_results_in_same_folder(config_filename: str, results_filename: str):
        """Verificar que ambos archivos están en la misma carpeta."""
        try:
            # Buscar ambos archivos
            for filename in [config_filename, results_filename]:
                results = service.files().list(
                    q=f"name='{filename}' and trashed=false",
                    fields="files(id, name, parents, webViewLink)"
                ).execute()
                
                files = results.get('files', [])
                if files:
                    file_info = files[0]
                    parents = file_info.get('parents', ['raíz'])
                    folder = parents[0] if parents else 'raíz'
                    
                    print(f"📄 {filename}")
                    print(f"   └─ Carpeta: {folder}")
                    if 'webViewLink' in file_info:
                        print(f"   └─ Link: {file_info['webViewLink']}")
                    
        except Exception as e:
            print(f"⚠️ Error verificando archivos: {e}")

    # Obtener carpeta donde está el archivo de configuración
    print(f"🎯 Iniciando proceso de guardado...")
    print(f"📂 Buscando carpeta del archivo de configuración...")
    
    parent_folder_id = get_config_file_parent_folder()
    
    if parent_folder_id:
        print(f"✅ Guardará en la misma carpeta que la configuración")
    else:
        print(f"📁 Guardará en la raíz de Google Drive")
    
    # Intentar guardar en Google Drive
    results_filename = config['output_config']['results_filename']
    
    drive_success = save_results_to_drive_with_retry(results, results_filename, parent_folder_id, max_retries=3)
    
    # Guardar backup local siempre
    local_success = save_results_locally(results, results_filename)
    
    # Mostrar resultados del guardado
    if drive_success:
        print(f"\n🎉 ¡Análisis N Preguntas Completado Exitosamente!")
        print(f"✅ Guardado en Google Drive: {results_filename}")
        
        if local_success:
            print(f"✅ Backup local también disponible")
        
        # Verificar que ambos archivos están juntos
        print(f"\n📂 Verificando ubicación de archivos:")
        verify_config_and_results_in_same_folder(CONFIG_FILENAME, results_filename)
        
        print(f"\n📋 Próximos pasos:")
        print(f"1. Ve a la aplicación Streamlit")
        print(f"2. Navega a 'Resultados Análisis N Preguntas'")
        print(f"3. Busca el archivo: {results_filename}")
        print(f"4. Explora las visualizaciones y métricas detalladas")
        print(f"5. ✨ Los archivos están en la misma carpeta para fácil acceso")
        
    elif local_success:
        print(f"\n⚠️ Google Drive falló, pero se guardó backup local")
        print(f"💾 Archivo local: /content/{results_filename}")
        print(f"\n📋 Para usar en Streamlit:")
        print(f"1. Descarga el archivo desde Colab:")
        print(f"   - Ve a Files (📁) en el panel izquierdo")
        print(f"   - Busca: {results_filename}")
        print(f"   - Click derecho → Download")
        print(f"2. Sube a Google Drive en la MISMA carpeta que:")
        print(f"   - {CONFIG_FILENAME}")
        print(f"3. Ve a Streamlit y carga el archivo")
        
    else:
        print(f"\n❌ Error en todos los métodos de guardado")
        print(f"💡 Los resultados siguen disponibles en la variable 'results'")
        print(f"📋 Puedes intentar ejecutar esta celda nuevamente")
    
    # Mostrar resumen de resultados
    display_results_summary(results)
    
    # Información de descarga manual si es necesario
    if not drive_success:
        print(f"\n🔧 DESCARGA MANUAL:")
        print(f"Si necesitas descargar los resultados manualmente:")
        
        # Crear un archivo comprimido con metadatos
        try:
            summary_data = {
                'filename': results_filename,
                'config_filename': CONFIG_FILENAME,
                'target_folder': parent_folder_id or 'root',
                'execution_stats': results.get('execution_stats', {}),
                'model_ranking': [(model, metrics.get('composite_score', {}).get('mean', 0)) 
                                 for model, metrics in results.get('consolidated_metrics', {}).items()],
                'total_questions': len(results.get('individual_results', {})),
                'timestamp': results.get('execution_stats', {}).get('completed_at', 'Unknown')
            }
            
            with open('/content/results_summary.json', 'w') as f:
                json.dump(summary_data, f, indent=2)
            
            print(f"📄 Resumen disponible en: /content/results_summary.json")
            print(f"📁 Incluye información de la carpeta objetivo")
            
        except Exception as e:
            print(f"⚠️ No se pudo crear resumen: {e}")
    
    print(f"\n✅ Proceso de guardado completado")